# Inference with the trained Word2Num Model

In [ ]:
import torch

In [ ]:
!pip install bpemb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 7.0 MB/s 


In [ ]:
torch.cuda.is_available()

True

In [ ]:
import re
from bpemb import BPEmb

from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
from torch.nn.utils.rnn import pad_sequence
import math
import numpy as np

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
BPEMB_HI = BPEmb(lang="hi", vs=25000, dim=100)
SRC_BPE_WEIGHTS = torch.tensor(BPEMB_HI.vectors)
SRC_VOCAB_SIZE = SRC_BPE_WEIGHTS.shape[0]
TGT_VOCAB_SIZE = 14
EMB_SIZE = 100
NHEAD = 4
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3



def row_processer_inp(input):
    X = input
    return re.sub(',', '', re.sub('-', ' ', X.strip()))

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int=None, emb_size=None, weights=None):
        super(TokenEmbedding, self).__init__()
        if weights is not None:            
            self.embedding = nn.Embedding.from_pretrained(weights)
            self.emb_size = weights.shape[1]
        else:
            self.embedding = nn.Embedding(vocab_size, emb_size)
            self.emb_size = emb_size
    
    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network 
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1,
                 src_bpe_weights = None):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        if src_bpe_weights is not None:
            self.src_tok_emb = TokenEmbedding(weights=src_bpe_weights)
        else:
            self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None, 
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

def token_transform(x):
    return BPEMB_HI.encode_ids(x)

def tensor_transform_src(token_ids):
    return torch.tensor(token_ids)

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        num = 0
        for transform in transforms:
            if num != 0:
                txt_input = [item for sublist in txt_input for item in sublist]
                a = np.array(txt_input).reshape(len(txt_input),1)
                txt_input = list(map(list, a))
                    
            txt_input = transform(txt_input)
            num = 1
        return txt_input
    return func

# function to generate output sequence using greedy algorithm 
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

# %%
# actual function to translate input sentence into target language
class NumberModel:


  
    def __init__(self, PATH):
        #Load the model here

        self.transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, 
                                    NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM, src_bpe_weights=SRC_BPE_WEIGHTS)
        self.transformer.load_state_dict(torch.load(PATH))
        self.transformer.to(DEVICE)
        

    def translate(self, model: torch.nn.Module, src_sentence: str):
        idx2class = dict(zip(list(range(14)), ['<unk>', '<pad>', '<bos>', '<eos>', '7', '2', '5', '4', '8', '3', '0', '1', '9', '6']))
        get_cls_preds = lambda x: [idx2class[i] for i in x]
        model.eval()

        text_transform = sequential_transforms(
                        token_transform, #Tokenization
                        tensor_transform_src) 
        src = text_transform(src_sentence.split()).view(-1, 1)
        num_tokens = src.shape[0]
        src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
        tgt_tokens = greedy_decode(
            model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
        return "".join(get_cls_preds(tgt_tokens.cpu().numpy())).replace("<bos>", "").replace("<eos>", "")


    def inference(self, text):
        #Call the inference model here
        digits = self.translate(self.transformer, text)
        return digits

In [ ]:
PATH = "/content/model_emb.pth"

text = "पाँच सौ अट्ठारह "
number_model = NumberModel(PATH)
text = number_model.inference(text)
print(text)

518


In [ ]:
import csv
def read_data(data):
    # open .tsv file
    with open(data, 'r', encoding="utf-8") as file:
        tsv_file = csv.reader(file, delimiter="\t")
        X_train = []
        y_train = []
        for line in tsv_file:
            X_train.append(line[0])
            y_train.append(line[1])

    return X_train, y_train

In [ ]:
from tqdm import tqdm
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, 
                                    NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM, src_bpe_weights=SRC_BPE_WEIGHTS)
transformer.load_state_dict(torch.load("model_emb.pth"))
transformer.to(DEVICE)

X_test, y_test = read_data("benchmark.tsv")

cnt_corr = 0
for i in tqdm(range(len(X_test))):
    pred = translate(transformer, row_processer_inp(X_test[i]))
    true = y_test[i]
    if pred == true:
        cnt_corr += 1
    else:
        print(X_test[i])
        print(pred)
        print(true)

print("Accuracy on test set: ", cnt_corr/len(X_test))



 86%|████████▌ | 172/200 [00:04<00:00, 40.42it/s]

नौ दो पाँच सात सौ छः छः
9257066
92570066


100%|██████████| 200/200 [00:05<00:00, 38.83it/s]

Accuracy on test set:  0.995
